In [2]:
import os
import random
import metpy
import datetime
import s3fs
import numpy as np
import pandas as pd
import xarray as xr
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from pyresample import geometry, grid
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from skimage.filters import laplace
from skimage.filters import unsharp_mask
from skimage.transform import resize
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

fs = s3fs.S3FileSystem(anon = True)
model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")
random.seed(42)

/sharedData2/nshakoor/.conda/envs/fires/lib/python3.7/site-packages/pyresample/bilinear/__init__.py:49: UserWarning: XArray and/or zarr not found, XArrayBilinearResampler won't be available.
  warnings.warn("XArray and/or zarr not found, XArrayBilinearResampler won't be available.")


In [3]:
def find_npy_file(path, file, band):
    """
    Returns the string name of another file from the same time for a given GOES band npy file,
    or None if there is no such file in the given directory.
    
    Parameter path: The path where the returned file is located in
    Precondition: path is a string to a directory relative to the current one, with .npy files
    
    Parameter file: A file from the same time the returned file should have
    Precondition: file is a string with the name of the original file, and includes the path
    
    Parameter band: GOES band the returned file should be from
    Precondition: band is a string of length 2
    """
    for x in os.listdir(path):
        if x[19:21] == band and file[23:37] == x[27:41]:
            return x

In [8]:
def control_img(img):
    """
    Returns a dictionary of errors for the given file without sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to evaluate the error of
    Precondition: file is a string
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    control = resize(smallimg, (height, width))
    return control

def laplace_sharpening_img(img):
    """
    Returns a dictionary of errors for the given file using laplace sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    laplace_edges = laplace(blurryimg)
    sharpimg = blurryimg + laplace_edges
    return sharpimg

def unsharpmask_sharpening_img(img):
    """
    Returns a dictionary of errors for the given file using unsharpmask sharpening. To normalize images,
    each image is divided by its maximum value, and then multiplied by the same value after sharpening
    is completed.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a str
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    sharpimg = unsharp_mask(blurryimg/blurryimg.max(), radius=1, amount=1)*blurryimg.max()
    return sharpimg

def update_dict_img(img07, img14, data1, data2, data3, data4):
    """
    Updates the dictionary with sharpening errors of a given file.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    
    Parameter data: Dictionary containing sharpening errors
    Precondition: data is a dict
    """
    m = (330-210)/(340-295)
    b = 210 - m*295
    fire_mask = (img14) < (m*img07 + b)
    
    data07 = control_img(img07)
    data14 = control_img(img14)
    fires = (data14) < (m*data07 + b)
    data1[0].append(len(fire_threshold(data07, data14)))
    data1[1].append(dice_score(fire_mask, fires))
    
    data07 = laplace_sharpening_img(img07)
    data14 = laplace_sharpening_img(img14)
    fires = (data14) < (m*data07 + b)
    data2[0].append(len(fire_threshold(data07, data14)))
    data2[1].append(dice_score(fire_mask, fires))
    
    data07 = unsharpmask_sharpening_img(img07)
    data14 = unsharpmask_sharpening_img(img14)
    fires = (data14) < (m*data07 + b)
    data3[0].append(len(fire_threshold(data07, data14)))
    data3[1].append(dice_score(fire_mask, fires))
    
#     data07 = esrgan_sharpening_img(img07)
#     data14 = esrgan_sharpening_img(img14)
#     data4.append(len(fire_threshold(data07, data14)))
    
def fire_threshold(img07, img14):
    m = (330-210)/(340-295)
    b = 210 - m*295
    points = []
    
    fire_mask = (img14) < (m*img07 + b)
    points = np.array(fire_mask.nonzero()).T
    
    for point in points:
        save = point[0]
        point[0] = point[1]
        point[1] = 500 - save
        
    return points

def dice_score(x, y):
    numerator = 2*np.count_nonzero(np.logical_and(x, y))
    denominator = len(x) + len(y)
    return numerator/denominator

In [5]:
def load_image(array):
    """
    Returns loaded .npy file.
    
    Parameter path: Path to load .npy file from
    Precondition: path is a string
    """
    result = np.zeros((array.shape[0], array.shape[1], 3))
    result[:,:,0]= array
    result[:,:,1]= array
    result[:,:,2]= array
    return result

def preprocess_image(array):
    """
    Returns preprocessed input array.
    
    Parameter array: array to preprocess
    Precondition: array is a numpy array
    """
    hr_image = array
    hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 2) * 2
    hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
    hr_image = tf.cast(hr_image, tf.float32)
    return tf.expand_dims(hr_image, 0)

def downscale_image(image):
    """
    Returns low resolution image after scaling down input image using nearest neighbor downsampling.

    Parameter image: 3D of 4D tensor of preprocessed image
    Precondition: image is a tensor
    """
    height, width = image.shape
    result = np.zeros((image.shape[0]//2, image.shape[1]//2, 3))
    smallimg = resize(image, (round(height/2), round(width/2)))
    result[:,:,0] = smallimg
    result[:,:,1] = smallimg
    result[:,:,2] = smallimg
    lr_image = tf.expand_dims(result, 0)
    lr_image = tf.cast(lr_image, tf.float32)
    return lr_image

def esrgan_sharpening(image):
    """
    Returns dictionary containing errors after using the ESRGAN model.
    
    Parameter path: path of image not including file name
    Precondition: path is a string
    
    Parameter file: file of the image
    Precondition: file is a string of a .npy file
    """
    hr_image = preprocess_image(load_image(image))
    lr_image = downscale_image(image)
    fake_image = model(lr_image)
    fake_image = tf.squeeze(fake_image)

    hr_image = tf.squeeze(hr_image).numpy()
    lr_image = tf.squeeze(lr_image).numpy()
    lr_image = resize(lr_image, (hr_image.shape[0], hr_image.shape[1], 3)).ravel()
    fake_image = resize(fake_image.numpy(), (hr_image.shape[0], hr_image.shape[1], 3)).ravel()
    hr_image = hr_image.ravel()

    control_mae = mean_absolute_error(hr_image, lr_image)
    control_rmse = mean_squared_error(hr_image, lr_image, squared=False)

    mae = mean_absolute_error(hr_image, fake_image)
    rmse = mean_squared_error(hr_image, fake_image, squared=False)
    return {'Control_MAE*': control_mae, 'Control_RMSE*': control_rmse, 'SR_MAE': mae, 'SR_RMSE': rmse}

def esrgan_sharpening_img(image):
    """
    Returns dictionary containing errors after using the ESRGAN model.
    
    Parameter path: path of image not including file name
    Precondition: path is a string
    
    Parameter file: file of the image
    Precondition: file is a string of a .npy file
    """
    hr_image = preprocess_image(load_image(image))
    lr_image = downscale_image(image)
    fake_image = model(lr_image)
    fake_image = tf.squeeze(fake_image)

    hr_image = tf.squeeze(hr_image).numpy()
    lr_image = tf.squeeze(lr_image).numpy()
    lr_image = resize(lr_image, (hr_image.shape[0], hr_image.shape[1], 3)).ravel()
    fake_image = resize(fake_image.numpy(), (hr_image.shape[0], hr_image.shape[1], 3)).ravel()
    hr_image = hr_image.ravel()

    return hr_image

# Error

In [6]:
filedates = set()
files = []

for file in os.listdir('../../GOES_Files/npy_files'):
    filedates.add(file[27:41])

for file in os.listdir('../../GOES_Files/clear_sky_mask'):
    if file[23:37] in filedates:
        files.append(file)

In [22]:
mean = []
std = []
path = '../../GOES_Files/npy_files/'
pathCM = '../../GOES_Files/clear_sky_mask/'
fires_initial = [[], []]
fires_final_control = [[], []]
fires_final_lp = [[], []]
fires_final_um = [[], []]
fires_final_ml = [[], []]

x = 0

while x < 10:
    try:
        CM = random.choice(files)
        files.remove(CM)

        file07 = find_npy_file('../../GOES_Files/npy_files', CM, '07')
        file14 = find_npy_file('../../GOES_Files/npy_files', CM, '14')

        img07 = np.load(path + file07)
        img14 = np.load(path + file14)

        mask = np.load(pathCM + CM)
        mask = mask.astype(bool)
#         img07[mask] = 0
#         img14[mask] = 0

        fires_initial.append(len(fire_threshold(img07, img14)))

        update_dict_img(img07, img14, fires_final_control, fires_final_lp, fires_final_um, fires_final_ml)

        x += 1

    except:
        pass

In [23]:
#mean number of fires for each image
print(np.mean(fires_initial[0]))
print(np.mean(fires_final_control[0]))
print(np.mean(fires_final_lp[0]))
print(np.mean(fires_final_um[0]))
# print(np.mean(fires_final_ml))

nan
0.0
0.3
0.0


In [24]:
#mean dice score
print(np.mean(fires_initial[1]))
print(np.mean(fires_final_control[1]))
print(np.mean(fires_final_lp[1]))
print(np.mean(fires_final_um[1]))
# print(np.mean(fires_final_ml))

nan
0.0
0.0004
0.0


In [22]:
fires_final_lp[1]

189